# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,thompson,55.7435,-97.8558,-19.91,71,0,7.72,CA,2025-02-05
1,1,mozarlandia,-14.7447,-50.5706,22.87,91,92,1.63,BR,2025-02-05
2,2,huaquillas,-3.4833,-80.2333,27.13,78,75,2.57,EC,2025-02-05
3,3,greenville,35.6127,-77.3663,14.01,80,6,6.26,US,2025-02-05
4,4,port-aux-francais,-49.3500,70.2167,5.88,98,100,9.41,TF,2025-02-05


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]
print(len(ideal_df))

ideal_df.dropna()

ideal_df.head()

17


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
105,105,lavrio,37.7145,24.0565,8.53,62,17,13.52,GR,2025-02-05
116,116,figuig (centre),32.1089,-1.2286,7.67,52,16,2.83,MA,2025-02-05
153,153,zaragoza,41.6561,-0.8773,8.25,63,0,5.66,ES,2025-02-05
182,182,bechar,31.6167,-2.2167,6.18,52,20,1.54,DZ,2025-02-05
223,223,hasaki,35.7333,140.8333,5.22,30,0,5.99,JP,2025-02-05


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name']=''

# Display sample data
# YOUR CODE HERE
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
105,lavrio,GR,37.7145,24.0565,62,
116,figuig (centre),MA,32.1089,-1.2286,52,
153,zaragoza,ES,41.6561,-0.8773,63,
182,bechar,DZ,31.6167,-2.2167,52,
223,hasaki,JP,35.7333,140.8333,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lon = row['Lng']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lavrio - nearest hotel: Nikolakakis Rooms Lavrio
figuig (centre) - nearest hotel: No hotel found
zaragoza - nearest hotel: Hotel Pilar Plaza
bechar - nearest hotel: Hôtel Madania
hasaki - nearest hotel: Choshi Plaza Hotel
dalbandin - nearest hotel: SF Al-dawood Restaurant
prince rupert - nearest hotel: Crest Hotel
shingu - nearest hotel: ホテルニューパレス
el calafate - nearest hotel: Hostería Las Avutardas
bahcelievler - nearest hotel: Holiday Inn Express Istanbul - Atakoy Metro
klaksvik - nearest hotel: Hotel Klaksvik
zanjan - nearest hotel: هتل قصر
cascais - nearest hotel: Cascais Hotel
shimoda - nearest hotel: ホテルウラガ
tinghir - nearest hotel: Riad Agraw
ceerigaabo - nearest hotel: Sanaag Hotel
at taraf - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
105,lavrio,GR,37.7145,24.0565,62,Nikolakakis Rooms Lavrio
116,figuig (centre),MA,32.1089,-1.2286,52,No hotel found
153,zaragoza,ES,41.6561,-0.8773,63,Hotel Pilar Plaza
182,bechar,DZ,31.6167,-2.2167,52,Hôtel Madania
223,hasaki,JP,35.7333,140.8333,30,Choshi Plaza Hotel
242,dalbandin,PK,28.8947,64.4101,33,SF Al-dawood Restaurant
245,prince rupert,CA,54.3161,-130.3201,21,Crest Hotel
273,shingu,JP,33.7333,135.9833,41,ホテルニューパレス
283,el calafate,AR,-50.3408,-72.2768,61,Hostería Las Avutardas
285,bahcelievler,TR,41.0023,28.8598,52,Holiday Inn Express Istanbul - Atakoy Metro


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
plot_4 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
# YOUR CODE HERE
plot_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)